In [7]:
from pathlib import Path
from vector_pipeline import settings

# Detect project root (one level above notebooks/)
project_root = Path("..").resolve() if Path(".").name == "notebooks" else Path(".").resolve()
print("Project root:", project_root)

# Override paths in settings to be absolute paths from project root
settings.DATA_PATH = project_root / "data" / "processed_product.pkl"
settings.CHROMA_DIR = project_root / "chroma_db"

print("DATA_PATH:", settings.DATA_PATH.resolve(), "-> exists:", settings.DATA_PATH.exists())
print("CHROMA_DIR:", settings.CHROMA_DIR.resolve())


Project root: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/notebooks
DATA_PATH: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/notebooks/data/processed_product.pkl -> exists: False
CHROMA_DIR: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/notebooks/chroma_db


In [6]:
from vector_pipeline.settings import DATA_PATH
from pathlib import Path

print("DATA_PATH as defined:", DATA_PATH)
print("Absolute path:", DATA_PATH.resolve())
print("Exists?", DATA_PATH.exists())

# Also list what's in ./data to see what you actually have
data_dir = Path("data")
print("\nContents of ./data directory:")
if data_dir.exists():
    for p in data_dir.iterdir():
        print(" -", p)
else:
    print("No 'data' directory found in current working dir.")


DATA_PATH as defined: data/processed_product.pkl
Absolute path: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/notebooks/data/processed_product.pkl
Exists? False

Contents of ./data directory:
No 'data' directory found in current working dir.


In [ ]:
from vector_pipeline.ingestion import build_or_load_vectorstore

print("Loading / building vectorstore...")
vs = build_or_load_vectorstore()
print("Vectorstore ready ✅")

Loading vectorstore (or building if missing)...
No existing DB found. Building Chroma vectorstore ...
Loading data from data/processed_product.pkl ...


FileNotFoundError: [Errno 2] No such file or directory: 'data/processed_product.pkl'

In [ ]:
def print_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))

In [ ]:
api_request = {
    "session_id": "S12345",
    "user_id": "U9001",
    "messages": [
        {"role": "user", "content": "Show me wireless headphones with noise cancelling"}
    ],
    "top_k": 4,
    "use_reranker": True
}

print("📥 Incoming API request:")
print_json(api_request)

In [ ]:
result = run_chat(
    messages=deepcopy(api_request["messages"]),
    k=api_request.get("top_k", 4),
    use_reranker=api_request.get("use_reranker", True)
)

api_response = {
    "status": "success",
    "session_id": api_request["session_id"],
    "user_id": api_request["user_id"],
    "answer": result["answer"],
    "messages": result["messages"],  # updated history
    "retrieved": result["retrieved"]
}

print("📤 API response:")
print_json(api_response)

In [ ]:
history = [
    {"role": "user", "content": "Recommend a gaming laptop"},
]

session_id = "S777"
user_id = "U007"

print("🧩 First turn — incoming:")
print_json({"session_id": session_id, "user_id": user_id, "messages": history})

result1 = run_chat(history)

print("\n🤖 First answer:")
print(result1["answer"])

print("\n📤 API response (turn 1):")
print_json({
    "status": "success",
    "session_id": session_id,
    "user_id": user_id,
    "answer": result1["answer"],
    "messages": result1["messages"],
    "retrieved": result1["retrieved"]
})


In [ ]:
history2 = result1["messages"] + [
    {"role": "user", "content": "What about something cheap but still good?"}
]

print("🧩 Second turn — incoming:")
print_json({"session_id": session_id, "user_id": user_id, "messages": history2})

result2 = run_chat(history2)

print("\n🤖 Second answer:")
print(result2["answer"])

print("\n📤 API response (turn 2):")
print_json({
    "status": "success",
    "session_id": session_id,
    "user_id": user_id,
    "answer": result2["answer"],
    "messages": result2["messages"],
    "retrieved": result2["retrieved"]
})


In [ ]:
print(f"📚 Retrieved chunks: {len(result2['retrieved'])}")

for i, chunk in enumerate(result2["retrieved"], start=1):
    print(f"\n--- Chunk {i} ---")
    print("Metadata:", chunk["metadata"])
    print("Snippet:", chunk["snippet"][:250], "...")


In [ ]:
def chat_once(session_id, user_id, user_input, history=None):
    if history is None:
        history = [{"role": "user", "content": user_input}]
    else:
        history = history + [{"role": "user", "content": user_input}]
    
    # simulate backend request
    req = {
        "session_id": session_id,
        "user_id": user_id,
        "messages": history,
    }
    
    res = run_chat(req["messages"])
    
    print("\n🗣️ User:", user_input)
    print("🤖 Bot:", res["answer"])
    return res["messages"]

# Example usage:
history = chat_once("S999", "UABC", "What is a good smartphone for photography?")
history = chat_once("S999", "UABC", "And something cheaper?", history)
history = chat_once("S999", "UABC", "Which one has best battery?", history)
